<a href="https://colab.research.google.com/github/xin-2001/taica_1132_GenAI/blob/main/HW8/113_2GenAI_HW8_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 【Demo07a】AI代理設計模式_Reflection

## 0. Reflection 的任務設計

### 🌟 任務說明：群組發言小幫手
常擔心在群組發言有口氣不佳，冒犯到其他人的問題嗎?使用這款AI發言小幫手，幫你修改你的發言內容，使之更禮貌，且流暢。

**🎯 流程說明：**
1. 使用者輸入要傳達的內容
2. `model_writer` 生成第一版文本（正式、禮貌、通順）
3. `model_reviewer` 檢查內容是否夠自然、口語化，並提供具體修改建議
4. `model_writer` 根據建議產出第二版
5. Gradio 呈現：三個欄位：第一版、建議、第二版

### 1.讀入金鑰

使用Groq

In [1]:
import os
from google.colab import userdata

In [2]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq_API')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [3]:
# Groq
!pip install groq

# OpenAI
!pip install openai

# Mistral
#!pip install mistralai

In [4]:
!pip install aisuite[all]

### 2.設置機器人模型

In [5]:
import aisuite as ai

1. 第一個機器人使用Groq的gemma2-9b-it
2. 第二個機器人使用Groq的llama3-70b-8192

In [6]:
provider_writer = "groq"
model_writer="gemma2-9b-it"

provider_reviewer = "groq"
model_reviewer = "llama3-70b-8192"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

In [7]:
system = ""
prompt = ""

In [26]:
def reply(system=system,
          prompt=prompt,
          provider=provider,
          model=model
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

###  3.定義兩個 Agent 的 system prompt

In [35]:
system_writer = "你是一位群組發言小幫手，擅長把使用者寫的內容，轉換成正式、禮貌、通順的發言內容，用於之事項傳遞或請求，並適當加入emoji，使語氣更自然親切。請用台灣習慣的中文回應。"
system_reviewer = "你是一位文案潤稿專家，請針對以下貼文給出具體修改建議。檢查檢查文案是否足夠正式、表達得自然、口語化，同時保留其禮貌和通順性，使其更貼近一般社群媒體的語境。並提醒使用台灣用語。請用台灣習慣的中文回應。"

In [36]:
def reflect_post(prompt):
    # Step 1: Writer 初稿
    first_version = reply(system_writer, prompt,
                          provider=provider_writer,
                          model=model_writer
                          )

    # Step 2: Reviewer 給建議
    suggestion = reply(system_reviewer, first_version,
                       provider=provider_reviewer,
                       model=model_reviewer
                       )

    # Step 3: Writer 再寫一次（根據建議）
    second_prompt = f"這是我剛剛寫的文案：\n{first_version}\n\n這是修改建議：\n{suggestion}\n\n請根據這些建議，幫我改得更生活化、更自然。"
    second_version = reply(system_writer, second_prompt,
                           provider=provider_writer,
                           model=model_reviewer
                           )

    return first_version, suggestion, second_version

### 4.用 Gradio 打造你的對話機器人 Web App!

In [17]:
!pip install gradio

In [18]:
import gradio as gr

In [37]:
with gr.Blocks() as demo:
    gr.Markdown("### 🤖 發言反思幫手（Reflection Agent）")
    user_input = gr.Textbox(label="請輸入你今天想發表的資料")
    btn = gr.Button("生成文本 & 修正建議")

    with gr.Row():
        out1 = gr.Textbox(label="🌟 第一版文本 (model_writer)")
        out2 = gr.Textbox(label="🧐 修改建議 (model_reviewer)")
        out3 = gr.Textbox(label="✨ 第二版文本 (model_writer 改寫)")

    btn.click(reflect_post, inputs=[user_input], outputs=[out1, out2, out3])

In [38]:
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://13ccc4f71099bb3c96.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 結果展現